In [0]:
SELECT
    al.borough,
    COUNT(*) AS total_listings,
    ROUND(AVG(TRY_CAST(f.price AS DECIMAL(18,2))), 2) AS avg_price,
    MIN(TRY_CAST(f.price AS DECIMAL(18,2))) AS min_price,
    MAX(TRY_CAST(f.price AS DECIMAL(18,2))) AS max_price
FROM public.fact_airbnb_listings f
JOIN public.dim_airbnb_location al
  ON f.geohash = al.geohash
WHERE al.borough IS NOT NULL
  AND f.price NOT IN ('NaN', 'nan', '', 'INF', '-INF')
  AND TRY_CAST(f.price AS DECIMAL(18,2)) IS NOT NULL
GROUP BY al.borough
ORDER BY total_listings DESC;

In [0]:
SELECT
    COALESCE(f.room_type, dp.room_type) AS room_type,
    al.borough,
    COUNT(*) AS listing_count,
    ROUND(AVG(TRY_CAST(f.price AS DECIMAL(18,2))), 2) AS avg_price
FROM public.fact_airbnb_listings f
JOIN public.dim_airbnb_location al
  ON f.geohash = al.geohash
LEFT JOIN public.dim_airbnb_property dp
  ON f.listing_id = dp.listing_id
WHERE al.borough IS NOT NULL
  AND f.price NOT IN ('NaN', 'nan', '', 'INF', '-INF')
  AND TRY_CAST(f.price AS DECIMAL(18,2)) IS NOT NULL   -- ignore rows that can’t be cast to numbers
GROUP BY COALESCE(f.room_type, dp.room_type), al.borough
ORDER BY listing_count DESC;

In [0]:
SELECT
    al.neighbourhood_name AS neighbourhood,
    al.borough,
    COUNT(*) AS listings,
    ROUND(AVG(f.price), 2) AS avg_price,
    ROUND(SUM(COALESCE(f.estimated_revenue_l365d, 0)), 0) AS est_total_revenue_l365d
FROM public.fact_airbnb_listings f
JOIN public.dim_airbnb_location al
  ON f.geohash = al.geohash
WHERE f.price NOT IN ('NaN', 'nan', '', 'INF', '-INF') 
GROUP BY al.neighbourhood_name, al.borough
ORDER BY listings DESC
LIMIT 20;